LSTM-based anomaly detection

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

Implementing a LSTM cell from scratch in NumPy.

In [2]:
# Now we have loaded data
df = pd.read_csv('behavioral_features.csv')
print(f"Dataset loaded: {df.shape[0]:,} rows, {df.shape[1]} columns")

# Getting sample user data
sample_user = df['user'].unique()[0]
user_data = df[df['user'] == sample_user].copy()
print(f"\nUsing user '{sample_user}' with {len(user_data)} days of data for verification.")

# Define features
FEATURE_COLS = ['logins', 'off_hour_logins', 'file_ops', 'files_copied',
                'usb_events', 'http_uploads', 'emails_sent']

# Normalize features
features = user_data[FEATURE_COLS].values.astype(np.float32)
scaler = MinMaxScaler()
features_normalized = scaler.fit_transform(features)

# Verification
X_verify = features_normalized[:7]
print(f"Verification sequence shape: {X_verify.shape}")
print(f"\nNormalized data (first 3 rows):\n{X_verify[:3]}")

Dataset loaded: 1,394,010 rows, 10 columns

Using user 'KAS2029' with 479 days of data for verification.
Verification sequence shape: (7, 7)

Normalized data (first 3 rows):
[[0.  0.  0.  0.  0.  0.  0.5]
 [0.  0.  0.  0.  0.  0.  1. ]
 [0.  0.  0.  0.  0.  0.  1. ]]


In [3]:
# Creating PyTorch LSTM Cell
input_size = 7
hidden_size = 4

lstm_cell = nn.LSTMCell(input_size, hidden_size)

# Extract weights and biases
W_ii, W_if, W_ig, W_io = lstm_cell.weight_ih.data.chunk(4, 0)
W_hi, W_hf, W_hg, W_ho = lstm_cell.weight_hh.data.chunk(4, 0)
b_ii, b_if, b_ig, b_io = lstm_cell.bias_ih.data.chunk(4, 0)
b_hi, b_hf, b_hg, b_ho = lstm_cell.bias_hh.data.chunk(4, 0)

# Convert to NumPy
weights_np = {
    'W_ii': W_ii.numpy(), 'W_if': W_if.numpy(), 'W_ig': W_ig.numpy(), 'W_io': W_io.numpy(),
    'W_hi': W_hi.numpy(), 'W_hf': W_hf.numpy(), 'W_hg': W_hg.numpy(), 'W_ho': W_ho.numpy(),
    'b_ii': b_ii.numpy(), 'b_if': b_if.numpy(), 'b_ig': b_ig.numpy(), 'b_io': b_io.numpy(),
    'b_hi': b_hi.numpy(), 'b_hf': b_hf.numpy(), 'b_hg': b_hg.numpy(), 'b_ho': b_ho.numpy()
}

print(f"Extracted weights from PyTorch LSTM cell")
print(f"Weight shapes - W_ii: {weights_np['W_ii'].shape}, W_hi: {weights_np['W_hi'].shape}")

Extracted weights from PyTorch LSTM cell
Weight shapes - W_ii: (4, 7), W_hi: (4, 4)


In [4]:
# ManualLSTM Class
class ManualLSTM:
    """NumPy implementation of LSTM cell."""

    def __init__(self, weights_dict):
        self.w = weights_dict

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def forward(self, x_seq, h_0=None, c_0=None):
        """Forward pass through sequence."""
        seq_len = x_seq.shape[0]
        hidden_size = self.w['W_hi'].shape[0]

        h = h_0 if h_0 is not None else np.zeros(hidden_size, dtype=np.float32)
        c = c_0 if c_0 is not None else np.zeros(hidden_size, dtype=np.float32)

        outputs = []
        for t in range(seq_len):
            x_t = x_seq[t]

            # Input gate
            i_t = self.sigmoid(x_t @ self.w['W_ii'].T + self.w['b_ii'] +
                              h @ self.w['W_hi'].T + self.w['b_hi'])
            # Forget gate
            f_t = self.sigmoid(x_t @ self.w['W_if'].T + self.w['b_if'] +
                              h @ self.w['W_hf'].T + self.w['b_hf'])
            # Cell gate
            g_t = np.tanh(x_t @ self.w['W_ig'].T + self.w['b_ig'] +
                         h @ self.w['W_hg'].T + self.w['b_hg'])
            # Output gate
            o_t = self.sigmoid(x_t @ self.w['W_io'].T + self.w['b_io'] +
                              h @ self.w['W_ho'].T + self.w['b_ho'])

            # Update cell state and hidden state
            c = f_t * c + i_t * g_t
            h = o_t * np.tanh(c)
            outputs.append(h.copy())

        return np.array(outputs), h, c

ManualLSTM class is now defined



In [5]:
# Verify Manual vs PyTorch Output
# PyTorch forward pass
X_torch = torch.tensor(X_verify, dtype=torch.float32)
h_torch = torch.zeros(hidden_size)
c_torch = torch.zeros(hidden_size)

torch_outputs = []
for t in range(X_torch.shape[0]):
    h_torch, c_torch = lstm_cell(X_torch[t:t+1], (h_torch.unsqueeze(0), c_torch.unsqueeze(0)))
    h_torch = h_torch.squeeze(0)
    c_torch = c_torch.squeeze(0)
    torch_outputs.append(h_torch.detach().numpy().copy())
torch_outputs = np.array(torch_outputs)

# Manual NumPy forward pass
manual_lstm = ManualLSTM(weights_np)
manual_outputs, _, _ = manual_lstm.forward(X_verify)

# Verification
match = np.allclose(manual_outputs, torch_outputs, atol=1e-6)
max_diff = np.max(np.abs(manual_outputs - torch_outputs))

print(f"PyTorch output shape: {torch_outputs.shape}")
print(f"Manual output shape: {manual_outputs.shape}")
print(f"\n np.allclose(manual_output, torch_output) = {match}")
print("\n NumPy LSTM matches PyTorch exactly")

PyTorch output shape: (7, 4)
Manual output shape: (7, 4)

 np.allclose(manual_output, torch_output) = True

 NumPy LSTM matches PyTorch exactly


Data Pipeline


In [6]:
# Spilt
all_users = df['user'].unique()
np.random.shuffle(all_users)

split_idx = int(len(all_users) * 0.8)
train_users = all_users[:split_idx]
test_users = all_users[split_idx:]

train_df = df[df['user'].isin(train_users)].copy()
test_df = df[df['user'].isin(test_users)].copy()

print(f"Total unique users: {len(all_users)}")
print(f"Train users: {len(train_users)} ({len(train_users)/len(all_users)*100:.1f}%)")
print(f"Test users: {len(test_users)} ({len(test_users)/len(all_users)*100:.1f}%)")
print(f"\nTrain samples: {len(train_df):,}")
print(f"Test samples: {len(test_df):,}")

Total unique users: 4000
Train users: 3200 (80.0%)
Test users: 800 (20.0%)

Train samples: 1,114,121
Test samples: 279,889


In [7]:
# Fit on train users only
scaler = MinMaxScaler()
scaler.fit(train_df[FEATURE_COLS])

train_scaled = scaler.transform(train_df[FEATURE_COLS]).astype(np.float32)
test_scaled = scaler.transform(test_df[FEATURE_COLS]).astype(np.float32)

print(f"Scaler fitted on training data only")
print(f"Train scaled shape: {train_scaled.shape}")
print(f"Test scaled shape: {test_scaled.shape}")

Scaler fitted on training data only
Train scaled shape: (1114121, 7)
Test scaled shape: (279889, 7)


In [8]:
# Windowing
WINDOW_SIZE = 7

def create_sequences_fast(data_df, scaled_data, window_size=7):
    """Create sequences per user using sliding window."""
    X_list, y_list = [], []
    users = data_df['user'].unique()

    start_idx = 0
    for user in users:
        user_len = len(data_df[data_df['user'] == user])
        user_data = scaled_data[start_idx:start_idx + user_len]

        if len(user_data) > window_size:

            windows = np.lib.stride_tricks.sliding_window_view(user_data, (window_size, user_data.shape[1]))
            windows = windows.squeeze(1)

            X_user = windows[:-1]  # All windows except last
            y_user = user_data[window_size:]  # Next-day targets

            X_list.append(X_user)
            y_list.append(y_user)

        start_idx += user_len

    return np.concatenate(X_list), np.concatenate(y_list)

# Sort data by user
train_df_sorted = train_df.sort_values(['user', 'day']).reset_index(drop=True)
test_df_sorted = test_df.sort_values(['user', 'day']).reset_index(drop=True)

# Re-scale after sorting
train_scaled = scaler.transform(train_df_sorted[FEATURE_COLS]).astype(np.float32)
test_scaled = scaler.transform(test_df_sorted[FEATURE_COLS]).astype(np.float32)

# Create sequences
X_train, y_train = create_sequences_fast(train_df_sorted, train_scaled, WINDOW_SIZE)
X_test, y_test = create_sequences_fast(test_df_sorted, test_scaled, WINDOW_SIZE)

print(f"Training sequences: X={X_train.shape}, y={y_train.shape}")
print(f"Test sequences: X={X_test.shape}, y={y_test.shape}")

Training sequences: X=(1091721, 7, 7), y=(1091721, 7)
Test sequences: X=(274289, 7, 7), y=(274289, 7)


In [9]:
# DataLoader
BATCH_SIZE = 2048

# Split training into train/val (80/20)
val_split = int(len(X_train) * 0.8)
X_train_final, X_val = X_train[:val_split], X_train[val_split:]
y_train_final, y_val = y_train[:val_split], y_train[val_split:]

# Create DataLoaders
train_dataset = TensorDataset(torch.tensor(X_train_final), torch.tensor(y_train_final))
val_dataset = TensorDataset(torch.tensor(X_val), torch.tensor(y_val))
test_dataset = TensorDataset(torch.tensor(X_test), torch.tensor(y_test))

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"\nDataLoaders created:")
print(f"  Train: {len(train_loader)} batches ({len(X_train_final):,} samples)")
print(f"  Val: {len(val_loader)} batches ({len(X_val):,} samples)")
print(f"  Test: {len(test_loader)} batches ({len(X_test):,} samples)")
print(f"  Batch size: {BATCH_SIZE}")


DataLoaders created:
  Train: 427 batches (873,376 samples)
  Val: 107 batches (218,345 samples)
  Test: 134 batches (274,289 samples)
  Batch size: 2048


Now we try to find the optimal architecture.

In [10]:
# Define  Sequence Model
class SequenceModel(nn.Module):
    """ RNN model supporting LSTM and GRU."""

    def __init__(self, input_size, hidden_size, num_layers, dropout, rnn_type='LSTM'):
        super().__init__()
        self.rnn_type = rnn_type
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Dynamically select RNN type
        rnn_class = nn.LSTM if rnn_type == 'LSTM' else nn.GRU
        self.rnn = rnn_class(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            batch_first=True
        )
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Last timestep
        return out

SequenceModel class is now defined



In [11]:
# Architecture Configurations
CONFIGS = {
    'Baseline_LSTM': {
        'rnn_type': 'LSTM', 'hidden_size': 64, 'num_layers': 1, 'dropout': 0.0,
    },
    'GRU': {
        'rnn_type': 'GRU', 'hidden_size': 64, 'num_layers': 1, 'dropout': 0.0,
    },
    'Deep_stack_LSTM': {
        'rnn_type': 'LSTM', 'hidden_size': 64, 'num_layers': 2, 'dropout': 0.2,
    },
    'Lightweight_LSTM': {
        'rnn_type': 'LSTM', 'hidden_size': 16, 'num_layers': 1, 'dropout': 0.0,
    },
    'Large_LSTM': {
        'rnn_type': 'LSTM', 'hidden_size': 128, 'num_layers': 1, 'dropout': 0.3,

    }
}

print(" Architectures Defined:")
for name, cfg in CONFIGS.items():
    print(f"  {name}: {cfg['rnn_type']}, H={cfg['hidden_size']}, L={cfg['num_layers']}, D={cfg['dropout']}")

 Architectures Defined:
  Baseline_LSTM: LSTM, H=64, L=1, D=0.0
  GRU: GRU, H=64, L=1, D=0.0
  Deep_stack_LSTM: LSTM, H=64, L=2, D=0.2
  Lightweight_LSTM: LSTM, H=16, L=1, D=0.0
  Large_LSTM: LSTM, H=128, L=1, D=0.3


In [12]:
# Training Function
def train_model(config_name, config, train_loader, val_loader, epochs=3):
    """Train a model and return results."""
    print(f"Training: {config_name}")
    model = SequenceModel(
        input_size=7,
        hidden_size=config['hidden_size'],
        num_layers=config['num_layers'],
        dropout=config['dropout'],
        rnn_type=config['rnn_type']
    )

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    train_losses, val_losses = [], []

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item()
        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(f"   Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    final_val_loss = val_losses[-1]
    print(f" Final Validation Loss: {final_val_loss:.6f}")

    return {
        'model_state': model.state_dict(),
        'final_val_loss': final_val_loss,
        'train_losses': train_losses,
        'val_losses': val_losses
    }

In [13]:

EPOCHS = 3
results = {}

for config_name, config in CONFIGS.items():
    result = train_model(config_name, config, train_loader, val_loader, epochs=EPOCHS)
    results[config_name] = result['final_val_loss']
    # Storing full results
    CONFIGS[config_name]['model_state'] = result['model_state']

print("Results: ")
for name, loss in sorted(results.items(), key=lambda x: x[1]):
    print(f"  {name}: {loss:.6f}")

Training: Baseline_LSTM
   Epoch 1/3 - Train Loss: 0.002767, Val Loss: 0.001408
   Epoch 2/3 - Train Loss: 0.001366, Val Loss: 0.001379
   Epoch 3/3 - Train Loss: 0.001354, Val Loss: 0.001371
 Final Validation Loss: 0.001371
Training: GRU
   Epoch 1/3 - Train Loss: 0.002600, Val Loss: 0.001407
   Epoch 2/3 - Train Loss: 0.001366, Val Loss: 0.001369
   Epoch 3/3 - Train Loss: 0.001353, Val Loss: 0.001367
 Final Validation Loss: 0.001367
Training: Deep_stack_LSTM
   Epoch 1/3 - Train Loss: 0.002846, Val Loss: 0.001438
   Epoch 2/3 - Train Loss: 0.001417, Val Loss: 0.001380
   Epoch 3/3 - Train Loss: 0.001372, Val Loss: 0.001362
 Final Validation Loss: 0.001362
Training: Lightweight_LSTM
   Epoch 1/3 - Train Loss: 0.004892, Val Loss: 0.001632
   Epoch 2/3 - Train Loss: 0.001469, Val Loss: 0.001439
   Epoch 3/3 - Train Loss: 0.001395, Val Loss: 0.001403
 Final Validation Loss: 0.001403
Training: Large_LSTM
   Epoch 1/3 - Train Loss: 0.002288, Val Loss: 0.001390
   Epoch 2/3 - Train Loss: 0

Model selection

In [16]:
# Selecting Best model
best_config_name = min(results, key=results.get)
best_val_loss = results[best_config_name]

print(f"\n{best_config_name} configuration is the best model.")
print(f"Final Validation Loss is: {best_val_loss:.6f}")


Large_LSTM configuration is the best model.
Final Validation Loss is: 0.001355


In [17]:
best_cfg = CONFIGS[best_config_name]

best_model = SequenceModel(
    input_size=7,
    hidden_size=best_cfg['hidden_size'],
    num_layers=best_cfg['num_layers'],
    dropout=best_cfg['dropout'],
    rnn_type=best_cfg['rnn_type']
)

best_model.load_state_dict(best_cfg['model_state'])
best_model.eval()

print(f"\nRe-instantiated {best_config_name} model.")
print(f"   Type: {best_cfg['rnn_type']}")
print(f"   Hidden Size: {best_cfg['hidden_size']}")
print(f"   Layers: {best_cfg['num_layers']}")
print(f"   Dropout: {best_cfg['dropout']}")



Re-instantiated Large_LSTM model.
   Type: LSTM
   Hidden Size: 128
   Layers: 1
   Dropout: 0.3


In [18]:
# Compute Threshold
def compute_errors(model, loader):
    """Compute reconstruction errors for a dataset."""
    model.eval()
    errors = []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            predictions = model(X_batch)
            batch_errors = torch.mean((predictions - y_batch) ** 2, dim=1)
            errors.extend(batch_errors.numpy())
    return np.array(errors)

# Compute errors on full training set
full_train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
train_errors = compute_errors(best_model, full_train_loader)

# Threshold at 95th percentile
threshold = np.percentile(train_errors, 95)

print(f"\n Threshold Computation")
print(f"Training Errors - Mean: {np.mean(train_errors):.6f}, Std: {np.std(train_errors):.6f}")
print(f"95th Percentile Threshold: {threshold:.6f}")


 Threshold Computation
Training Errors - Mean: 0.001339, Std: 0.004691
95th Percentile Threshold: 0.006497


In [19]:
# Evaluate on Test Set
test_errors = compute_errors(best_model, test_loader)
anomalies = test_errors > threshold

print(f"Test Errors - Mean: {np.mean(test_errors):.6f}, Std: {np.std(test_errors):.6f}")
print(f"Anomalies Detected: {np.sum(anomalies):,} / {len(test_errors):,} ({np.mean(anomalies)*100:.2f}%)")

Test Errors - Mean: 0.001167, Std: 0.004240
Anomalies Detected: 12,129 / 274,289 (4.42%)
